# Demand side flexibility


In [1]:
using Pkg
Pkg.add("CSV")
Pkg.add("DataFrames")
Pkg.add("JuMP")
Pkg.add("Gurobi")
#Pkg.add("NLopt")

   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`
   Resolving package versions...
  No Changes to `~/.julia/environments/v1.9/Project.toml`
  No Changes to `~/.julia/environments/v1.9/Manifest.toml`


In [2]:
using CSV, DataFrames, JuMP, Gurobi

# Read the CSV data

In [3]:
#fixed_cost = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/fixed_cost.csv",DataFrame))
#variable_cost = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/variable_cost.csv",DataFrame))
#demand = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/demand.csv", DataFrame))
#Availability_matrix = Matrix(CSV.read("/Users/adrienguenard/Desktop/MIT/2.AeroAstro/IDS.522/Project/1.WIP/DSF/test_data/availability.csv", DataFrame));

In [4]:
fixed_cost = Matrix(CSV.read("fixed_cost.csv",DataFrame))
variable_cost = Matrix(CSV.read("variable_cost.csv",DataFrame))
demand = Matrix(CSV.read("demand.csv", DataFrame))
max_capacity_matrix = Matrix(CSV.read("max_capacity.csv", DataFrame))
Availability_matrix = Matrix(CSV.read("Availability.csv", DataFrame));

In [5]:
C_FC = [fixed_cost[1:2,1]; fixed_cost[5,1]]
C_FOM = [fixed_cost[1:2,2]; fixed_cost[5,2]]
C_VOM = variable_cost[:,1]
E_CO2 = variable_cost[:,2]
X_D = demand[1:1000]
max_capacity = max_capacity_matrix[:]
Availability = Availability_matrix[1:1000,1:2]

1000×2 Matrix{Float64}:
 0.00393802   0.53339
 0.0          0.513005
 0.0          0.518819
 0.0          0.481426
 0.0          0.475112
 0.0          0.469125
 0.0          0.468391
 0.0          0.453982
 0.0          0.411274
 0.0          0.376903
 0.0          0.356235
 0.0          0.375655
 0.0          0.403653
 ⋮            
 0.0          0.431348
 0.0          0.43697
 0.0          0.433476
 0.0          0.420068
 0.0          0.352161
 0.0          0.35399
 0.0          0.370464
 0.0          0.382713
 0.000379575  0.394844
 0.0332202    0.42464
 0.117423     0.459098
 0.229411     0.499756

In [6]:
println(C_FC)

[1.623186399e6, 666821.2034, 434537.234]


# Constant

In [7]:
eta = 0.8# Efficiency of the battery



0.8

# Model Functions

In [8]:
function cost_mix(P, X)
    CO2_price = 1000000 # price of the CO2 per ton
    # The generation for the fossil fuel is for the index 3 and 4 of X
    print(sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)))
    cost = sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1)) 
                sum(sum((C_VOM[j] + CO2_price * E_CO2[j])* X[t,j+2] for j in 1:2) for t in 1:size(X,1)) 
    return cost
end

cost_mix (generic function with 1 method)

# Creating the model

List of variable 

In [9]:
Power_generation_ls = 1:3 # Solar, Wind, Battery

fossil_fuel_ls = 1:2 # CCGT, Coal

nb_hours_ls = 1:size(X_D,1)

H = 2

2

In [10]:
model = Model(Gurobi.Optimizer)
println("Declare Model")
set_optimizer_attribute(model, "NonConvex", 2)
#model = Model(NLopt.Optimizer)
#set_optimizer_attribute(model, "algorithm", :LD_MMA)

#The installed power of the mix
@variable(model, P[Power_generation_ls] >= 0)

@constraint(model, Battery_power, P[3] >= 10000)
println("constraint Power")
#@constraint(model, Solar_power, P[1] <= max_capacity[1])
#@constraint(model, Wind_power, P[2] <= max_capacity[2])

#Generation of solar, wind, coal, CCGT
@variable(model, X_gen[nb_hours_ls, Power_generation_ls ] >= 0)

#State of charge of the battery
@variable(model, X_soc[nb_hours_ls] >=0)

# Discharge of the battery
@variable(model, X_dch[nb_hours_ls] >=0)

# Charge of the battery
@variable(model, X_ch[nb_hours_ls] >=0)

println("Objective function")
# The cost of the mix assuming a certain carbon cost
@objective(model, Min, sum((C_FC[i] + C_FOM[i])*P[i] for i in 1:size(P,1))) # to be changed


# Cannot produce more than the installed power
println("Installed power constraint")
@constraint(model, Solar_generation[t in nb_hours_ls], X_gen[t,1] <= P[1])
@constraint(model, Wind_generation[t in nb_hours_ls], X_gen[t,2] <= P[2])
@constraint(model, Battery[t in nb_hours_ls], X_gen[t,3] <= P[3])

# Cannot produce more than availability for solar and wind 
println("Availability Constraint")
@constraint(model, Solar_available[t in nb_hours_ls], X_gen[t,1] <= Availability[t,1]*P[1])
@constraint(model, Wind_available[t in nb_hours_ls], X_gen[t,2] <=  Availability[t,2]*P[2])

# Supply meets demand at each hours:
println("Supply meets demand constraint")
@constraint(model, Supply_demand[t in nb_hours_ls], sum(X_gen[t,i] for i in Power_generation_ls) >= X_D[t]);


Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Declare Model
Set parameter NonConvex to value 2
constraint Power
Objective function
Installed power constraint
Availability Constraint
Supply meets demand constraint


![Alt text](image.png)

Create the variable X_min = min(X_1, X_2, X_3)

In [11]:
M = 10000000000000

# X_1, X_2, X_3 = [eta*(sum(X_gen[t,i] for i in Power_generation_ls)- X_D[t]) , eta*P3], (1- X_soc[t])*P[5]* H]
@variable(model, z1[nb_hours_ls], Bin)
@variable(model, z2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Z_1 expression")
@constraint(model, z_1_1[t in nb_hours_ls], eta*P[3] - eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*z1[t])
@constraint(model, z_1_2[t in nb_hours_ls], -eta*P[3] + eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-z1[t]))


#Minimum between X_1 and X_2
@variable(model, min_X_1_X_2[nb_hours_ls])

println("Min X_1, X_2 expression")
@constraint(model, X_1_X_2_lower_1[t in nb_hours_ls], min_X_1_X_2[t] <= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]))
@constraint(model, X_1_X_2_lower_2[t in nb_hours_ls], min_X_1_X_2[t] <= eta*P[3])
@constraint(model, X_1_X_2_higher_1[t in nb_hours_ls], min_X_1_X_2[t] >= eta*(sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-z1[t]))
@constraint(model, X_1_X_2_higher_2[t in nb_hours_ls], min_X_1_X_2[t] >= eta*P[3] - M*z1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, z_2_1[t in nb_hours_ls], (1- X_soc[t])* P[3] * H - min_X_1_X_2[t] <=  M*z2[t])
@constraint(model, z_2_2[t in nb_hours_ls], - (1- X_soc[t]) * P[3] * H + min_X_1_X_2[t] <= M*(1-z2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_X_1_X_2_X_3[nb_hours_ls])

@constraint(model, X_1_X_2_X_3_lower_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= min_X_1_X_2[t])
@constraint(model, X_1_X_2_X_3_lower_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] <= (1- X_soc[t])*P[3]* H)
@constraint(model, X_1_X_2_X_3_higher_1[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= min_X_1_X_2[t] - M*(1-z2[t]))
@constraint(model, X_1_X_2_X_3_higher_2[t in nb_hours_ls], min_X_1_X_2_X_3[t] >= (1- X_soc[t])*P[3]* H - M*z2[t]);



Z_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_ch

In [12]:
# Need to put this conditions above
@variable(model, Z_X_gen_greater_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_greater_X_D_1[t in nb_hours_ls], sum(X_gen[t,i] for i in 1:2) - X_D[t] <= M*Z_X_gen_greater_X_D[t])
@constraint(model, Z_X_gen_greater_X_D_2[t in nb_hours_ls], -sum(X_gen[t,i] for i in 1:2) + X_D[t] <= M*(1-Z_X_gen_greater_X_D[t]))

#Expression of X_ch
@constraint(model, X_ch_expression[t in nb_hours_ls], X_ch[t]  == Z_X_gen_greater_X_D[t]*min_X_1_X_2_X_3[t]);

Expression of min of [X_gen[t]- X_D[t],  P[5], X_soc[t]*P3]* H]

In [13]:
# X_1, X_2, X_3 = [X_gen[t]- X_D[t]) , P3], X_soc[t])*P3]* H]
@variable(model, y1[nb_hours_ls], Bin)
@variable(model, y2[nb_hours_ls], Bin)

# z_1 equal 1 if eta*(x_g_t -x_d) is is greatter than eta*E_max*delta t
println("Y_1 expression")
@constraint(model, y_1_1[t in nb_hours_ls], P[3] - (sum(X_gen[t,i] for i in 1:2)- X_D[t])   <=  M*y1[t])
@constraint(model, y_1_2[t in nb_hours_ls], -P[3] + (sum(X_gen[t,i] for i in 1:2)- X_D[t]) <= M*(1-y1[t]))


#Minimum between X_1 and X_2
@variable(model, min_extra_prod_P_5[nb_hours_ls])

println("Min X_1, X_2 expression")
@constraint(model, min_extra_prod_P_5_lower_1[t in nb_hours_ls], min_extra_prod_P_5[t] <= sum(X_gen[t,i] for i in 1:2)- X_D[t])
@constraint(model, min_extra_prod_P_5_lower_2[t in nb_hours_ls], min_extra_prod_P_5[t] <= P[3])
@constraint(model, min_extra_prod_P_5_higher_1[t in nb_hours_ls], min_extra_prod_P_5[t] >= (sum(X_gen[t,i] for i in 1:2)- X_D[t]) - M*(1-y1[t]))
@constraint(model, min_extra_prod_P_5_higher_2[t in nb_hours_ls], min_extra_prod_P_5[t] >= P[3] - M*y1[t])


# z_2 equal 1 if min(X_1, X_2) is is greater than (1- x_soc[t])*h*E_max
println(" Take the min of min(X_1, X_2) and X_3")
@constraint(model, y_2_1[t in nb_hours_ls], X_soc[t]* P[3] * H - min_extra_prod_P_5[t] <=  M*y2[t])
@constraint(model, y_2_2[t in nb_hours_ls], - X_soc[t] * P[3] * H + min_extra_prod_P_5[t] <= M*(1-y2[t]))


# Constraint for the variable X_ch
println(" Take the overall min")
@variable(model, min_extra_prod_P5_soc[nb_hours_ls])

@constraint(model, min_extra_prod_P5_soc_lower_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= min_extra_prod_P_5[t])
@constraint(model, min_extra_prod_P5_soc_lower_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] <= X_soc[t]*P[3]* H)
@constraint(model, min_extra_prod_P5_soc_higher_1[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= min_extra_prod_P_5[t] - M*(1-y2[t]))
@constraint(model, min_extra_prod_P5_soc_higher_2[t in nb_hours_ls], min_extra_prod_P5_soc[t] >= X_soc[t]*P[3]* H - M*y2[t]);



Y_1 expression
Min X_1, X_2 expression
 Take the min of min(X_1, X_2) and X_3
 Take the overall min


Expression of X_dch

In [14]:
@variable(model, Z_X_gen_lower_X_D[nb_hours_ls], Bin)

@constraint(model, Z_X_gen_lower_X_D_1[t in nb_hours_ls], X_D[t] - sum(X_gen[t,i] for i in 1:2)  <= M*Z_X_gen_lower_X_D[t])
@constraint(model, Z_X_gen_lower_X_D_2[t in nb_hours_ls], -X_D[t] + sum(X_gen[t,i] for i in 1:2) <= M*(1-Z_X_gen_lower_X_D[t]))

#Expression of X_dch
# issue with this constraint
@constraint(model, X_dch_expression[t in nb_hours_ls], X_gen[t,3]  == Z_X_gen_lower_X_D[t]*min_extra_prod_P5_soc[t]);

Expression of state of charge

In [15]:
@constraint(model, SOC_t_1, X_soc[1] == 1)

@constraint(model, SOC_t[t in nb_hours_ls[2:end]], (X_soc[t]-X_soc[t-1])*H*P[3] == (X_ch[t-1]- X_dch[t-1]));

In [16]:
optimize!(model);

Set parameter NonConvex to value 2
Gurobi Optimizer version 10.0.2 build v10.0.2rc0 (mac64[arm])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 26002 rows, 16003 columns and 74558 nonzeros
Model fingerprint: 0xf011e307
Model has 10999 quadratic constraints
Variable types: 10003 continuous, 6000 integer (6000 binary)
Coefficient statistics:
  Matrix range     [1e-08, 1e+13]
  QMatrix range    [1e+00, 2e+00]
  QLMatrix range   [1e+00, 1e+13]
  Objective range  [4e+05, 2e+06]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+13]
  QRHS range       [1e+13, 1e+13]
         Consider reformulating model or setting NumericFocus parameter
         to avoid numerical issues.
Presolve added 7110 rows and 1550 columns
Presolve time: 0.16s
Presolved: 48099 rows, 24550 columns, 130526 nonzeros
Presolved model has 3998 SOS constraint(s)
Presolved model has 2996 bilinear constraint(s)
Variable types: 16550 continuou

In [17]:
objective_value(model)

LoadError: Result index of attribute MathOptInterface.ObjectiveValue(1) out of bounds. There are currently 0 solution(s) in the model.

In [ ]:
println(value.(P))
#println(value.(s1[:]))
println(value.(X_soc))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:3
And data, a 3-element Vector{Float64}:
 0.0
 4.934200031641337e6
 3.4924596548080444e-10
1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1000
And data, a 1000-element Vector{Float64}:
 1.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0


 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0

In [ ]:
#println(value.(X_gen))
X_gen_matrix = value.(X_gen)




2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, 1:1000
    Dimension 2, 1:3
And data, a 1000×3 Matrix{Float64}:
 0.0       4.57729e5  1.16415e-10
 0.0       4.48949e5  3.49246e-10
 0.0  444216.0        2.32831e-10
 0.0       4.40378e5  2.32831e-10
 0.0       4.42357e5  2.32831e-10
 0.0       4.50506e5  2.32831e-10
 0.0       4.66266e5  2.32831e-10
 0.0       4.69709e5  2.32831e-10
 0.0       4.70139e5  2.32831e-10
 0.0       4.76335e5  2.32831e-10
 0.0       4.8284e5   2.32831e-10
 0.0       4.89425e5  2.32831e-10
 0.0       4.94409e5  1.16415e-10
 ⋮                    
 0.0       5.15446e5  3.49246e-10
 0.0       5.39505e5  3.49246e-10
 0.0       5.68763e5  3.49246e-10
 0.0       5.89897e5  3.49246e-10
 0.0       5.99478e5  3.49246e-10
 0.0       6.01964e5  3.49246e-10
 0.0       6.05141e5  3.49246e-10
 0.0       6.02652e5  3.49246e-10
 0.0       5.96812e5  3.49246e-10
 0.0       5.87828e5  3.49246e-10
 0.0       5.85802e5  3.49246e-10
 0.0       5.91642e

In [ ]:
using Plots

@userplot StackedArea

# a simple "recipe" for Plots.jl to get stacked area plots
# usage: stackedarea(xvector, datamatrix, plotsoptions)
@recipe function f(pc::StackedArea)
    x, y = pc.args
    n = length(x)
    y = cumsum(y, dims=2)
    seriestype := :shape

    # create a filled polygon for each item
    for c=1:size(y,2)
        sx = vcat(x, reverse(x))
        sy = vcat(y[:,c], c==1 ? zeros(n) : reverse(y[:,c-1]))
        @series (sx, sy)
    end
end

a = X_gen_matrix[:,1]
b = X_gen_matrix[:,2]
c = X_gen_matrix[:,3]
sNames = ["a","b","c"]
x = nb_hours_ls

plotly()
stackedarea(x, [a b c], labels=reshape(sNames, (1,3)))
#genplot(x, X_D[:])

In [ ]:
P_3_matrix = value(P[3])*ones(size(nb_hours_ls))

println(P_3_matrix)
println("X gen")
X_gen_non_battery = sum(X_gen_matrix[:,i] for i =1:2) - X_D[:]
println("X soc")
X_soc_charge = (ones(size(X_soc))- value.(X_soc))* value(P[3]) * H

println("X ch")
X_charge = value.(X_ch)

plot(x, [P_3_matrix X_gen_non_battery X_soc_charge X_charge], label=["P[3]" "X_gen Non batt" "X possible charge" "X_ch"])


[3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080

 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080444e-10, 3.4924596548080

In [ ]:
println(value.(min_extra_prod_P5_soc))
println(value.(Z_X_gen_lower_X_D))

1-dimensional DenseAxisArray{Float64,1,...} with index sets:
    Dimension 1, 1:1000
And data, a 1000-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.